In [145]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [146]:
from scipy.io import loadmat
import os
import numpy as np
import pandas as pd
from quad import Quads #custom class for quad data
import tdt

In [147]:
basedir = '/home/danhan/code/data/quad_data'
animal = 'Diego'
date = '251118'
data_dir = f'{basedir}/{animal}/{date}'

In [148]:
bhv_list = [f'{data_dir}/{f}' for f in os.listdir(data_dir) if f.endswith('mat')]

In [149]:
if animal == 'Diego':
    subject = 'S1'
elif animal == 'Pancho':
    subject = 'S2'


with open(f'{basedir}/{animal}/conditions_quadrilaterals_{subject}.txt', 'r') as f:
    conditions = pd.read_csv(f, delimiter = '\t')

In [150]:
bhv_list[0]

'/home/danhan/code/data/quad_data/Diego/251118/251118_154252_finalquads3_Diego_2.mat'

In [151]:
dat = loadmat(bhv_list[0],simplify_cells=True)
quad = Quads(dat,conditions)
trial = 3
dat_trial = dat[f'Trial{trial}']

In [152]:
stim_names = quad.getListStimNames(trial)

In [153]:
stims_present, stims_binary = quad.getWhatStimEachPresentation(trial)

In [154]:
stim_dict_manual = {}
i = 0
for stim,err in zip(stims_present,stims_binary):
    stim_dict_manual[i] = (stim,err)
    i += 1

In [155]:
codes = [c for c in dat_trial['BehavioralCodes']['CodeNumbers'] if 102 <= c <= 131]

In [156]:
trial_record = dat['TrialRecord']

In [157]:
trial_record['User']['TrialData'][0].keys()
fnames = trial_record['User']['TrialData'][trial-1]['sample_filename']
errs = trial_record['User']['TrialData'][trial-1]['sample_error_code']
stim_dict_record = {}
i = 0
for stim,err in zip(fnames,errs):
    stim_dict_record[i] = (stim,err)
    i += 1

In [161]:
neural_dir = f'/home/danhan/freiwaldDrive/ltian/recordings/{animal}/{date}'
all_entries = os.listdir(neural_dir)
eventcodes_dir = f'{neural_dir}/{all_entries[0]}'
neural = tdt.read_block(eventcodes_dir,t1=0,t2=600)

read from t=0s to t=600s


In [185]:
quad.neural = neural

In [ ]:
neural.epocs.SMa1.data

array([  9.,  10.,  20., ..., 103.,  21.,  10.])

In [191]:
neural.epocs.SMa1

name:	'SMa1'
onset:	array([ 34.0856832 ,  34.11279872,  34.89652736, ..., 598.99871232,
       599.81377536, 599.8166016 ])
offset:	array([ 34.11279872,  34.89652736,  34.89796096, ..., 599.81377536,
       599.8166016 , 600.        ])
type:	'onset'
type_str:	'epocs'
data:	array([  9.,  10.,  20., ..., 103.,  21.,  10.])
dform:	np.uint32(4)
size:	10

In [196]:
dat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'MLConfig', 'Trial1', 'Trial2', 'Trial3', 'Trial4', 'Trial5', 'Trial6', 'Trial7', 'Trial8', 'Trial9', 'Trial10', 'Trial11', 'Trial12', 'Trial13', 'Trial14', 'Trial15', 'Trial16', 'Trial17', 'Trial18', 'Trial19', 'Trial20', 'Trial21', 'Trial22', 'Trial23', 'Trial24', 'Trial25', 'Trial26', 'Trial27', 'Trial28', 'Trial29', 'Trial30', 'Trial31', 'Trial32', 'Trial33', 'Trial34', 'Trial35', 'Trial36', 'Trial37', 'Trial38', 'Trial39', 'Trial40', 'Trial41', 'Trial42', 'Trial43', 'Trial44', 'Trial45', 'Trial46', 'Trial47', 'Trial48', 'Trial49', 'Trial50', 'Trial51', 'Trial52', 'Trial53', 'Trial54', 'Trial55', 'Trial56', 'Trial57', 'Trial58', 'Trial59', 'Trial60', 'Trial61', 'Trial62', 'Trial63', 'Trial64', 'Trial65', 'Trial66', 'Trial67', 'Trial68', 'Trial69', 'Trial70', 'Trial71', 'Trial72', 'Trial73', 'Trial74', 'Trial75', 'Trial76', 'Trial77', 'Trial78', 'Trial79', 'Trial80', 'Trial81', 'Trial82', 'Trial83', 'Trial84', 'Trial85', 'Trial8

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

In [211]:
quad = Quads(dat,conditions,neural)

In [212]:
quad.prettyBeh

,trial_ml2,stim_index,stim_presented,fixation_success_binary
0,1,0,"shape-random_version-reference_dil-6_rot-2,0,0",True
1,1,1,"shape-random_version-reference_dil-5_rot-3,0,0",True
2,1,2,"shape-random_version-reference_dil-1_rot-6,0,0",True
3,1,3,"shape-random_version-reference_dil-5_rot-2,0,0",True
4,1,4,"shape-random_version-reference_dil-3_rot-3,0,0",True
...,...,...,...,...
3116,89,3116,"shape-random_version-reference_dil-3_rot-3,0,0",True
3117,89,3117,"shape-random_version-reference_dil-1_rot-2,0,0",True
3118,89,3118,"shape-random_version-outlier1_dil-5_rot-1,0,0",True
3119,89,3119,"shape-random_version-reference_dil-1_rot-5,0,0",True


In [223]:
neural.epocs.SMa1

name:	'SMa1'
onset:	array([ 34.0856832 ,  34.11279872,  34.89652736, ..., 598.99871232,
       599.81377536, 599.8166016 ])
offset:	array([ 34.11279872,  34.89652736,  34.89796096, ..., 599.81377536,
       599.8166016 , 600.        ])
type:	'onset'
type_str:	'epocs'
data:	array([  9.,  10.,  20., ..., 103.,  21.,  10.])
dform:	np.uint32(4)
size:	10